In [2]:
import QuanticsGrids as QG     
# utilities for handling quantics representations
import TCIAlgorithms as TCIA   
# implementation of patching
using HubbardAtoms             
# exact results for the Hubbard atom
using SparseIR                 
# provides the MatsubaraFreq types used in the HubbardAtoms package
using Quantics                 
# high-level API for performing operations in QTT
using ITensors                 
# efficient tensor computations and tensor network calculations

In [3]:
function main(U, beta, ch, R, maxbonddim)
    grid, sites = setup(R)
    plainfuncs, quanticsfuncs = makeverts(U, beta, ch, grid)
    patchesfuncs = interpolateverts(quanticsfuncs, grid, maxbonddim, sites)
    pttfuncs, diagonal_sites = makevertsdiagonal(patchesfuncs, sites)
    phi_bse = calculatebse(pttfuncs, diagonal_sites, maxbonddim, sites)
    error = comparereference(phi_bse, plainfuncs, grid)
    return error
end;

In [4]:
function setup(R=4)
    N = 2^R
    grid = QG.InherentDiscreteGrid{3}(R, (-N + 1, -N + 1, -N);
        step=2, unfoldingscheme=:fused)

    sitesv = [Index(2, "v=$r") for r in 1:R]
    sitesv´ = [Index(2, "v´=$r") for r in 1:R]
    sitesw = [Index(2, "w=$r") for r in 1:R]
    sitesfused = collect.(zip(sitesv, sitesv´, sitesw))
    sites = (; sitesv, sitesv´, sitesw, sitesfused)

    return grid, sites
end;

In [5]:
function makeverts(U, beta, ch, grid)
    model = HubbardAtom(U, beta)

    matsubara(v, v´, w) = (FermionicFreq(v), FermionicFreq(v´), BosonicFreq(w))

    fq_full(v, v´, w) = real(full_vertex(ch, model, matsubara(v, v´, w)))
    fq_chi0(v, v´, w) = 1 / beta^2 * real(chi0(ch, model, matsubara(v, v´, w)))
    fq_gamma(v, v´, w) = real(gamma(ch, model, matsubara(v, v´, w)))
    
    plainfuncs = (; fq_full, fq_chi0, fq_gamma)

    fI_full = QG.quanticsfunction(Float64, grid, fq_full)
    fI_chi0 = QG.quanticsfunction(Float64, grid, fq_chi0)
    fI_gamma = QG.quanticsfunction(Float64, grid, fq_gamma)
    quanticsfuncs = (; fI_full, fI_chi0, fI_gamma)

    return plainfuncs, quanticsfuncs
end;

In [6]:
function interpolateverts(quanticsfuncs, grid, maxbonddim, sites)
    (; fI_full, fI_chi0, fI_gamma) = quanticsfuncs

    localdims = dim.(sites.sitesfused)
    projectable_full = TCIA.makeprojectable(Float64, fI_full, localdims)
    projectable_chi0 = TCIA.makeprojectable(Float64, fI_chi0, localdims)
    projectable_gamma = TCIA.makeprojectable(Float64, fI_gamma, localdims)

    initialpivots = [QG.origcoord_to_quantics(grid, 0)] # approximate center
    full_patches = TCIA.adaptiveinterpolate(projectable_full;
                                            maxbonddim, initialpivots)
    chi0_patches = TCIA.adaptiveinterpolate(projectable_chi0;
                                            maxbonddim, initialpivots)
    gamma_patches = TCIA.adaptiveinterpolate(projectable_gamma;
                                            maxbonddim, initialpivots)

    sitedims = [dim.(s) for s in sites.sitesfused]
    full_patches = reshape(full_patches, sitedims)
    chi0_patches = reshape(chi0_patches, sitedims)
    gamma_patches = reshape(gamma_patches, sitedims)

    patchesfuncs = (; full_patches, chi0_patches, gamma_patches)

    return patchesfuncs
end;

In [7]:
function makevertsdiagonal(patchesfuncs, sites)
    (; full_patches, chi0_patches, gamma_patches) = patchesfuncs
    (; sitesv, sitesv´, sitesw, sitesfused) = sites

    full_mps = TCIA.ProjMPSContainer(Float64, full_patches, sitesfused)
    chi0_mps = TCIA.ProjMPSContainer(Float64, chi0_patches, sitesfused)
    gamma_mps = TCIA.ProjMPSContainer(Float64, gamma_patches, sitesfused)

    sitesvv´_vec = [[v, v´] for (v, v´) in zip(sitesv, sitesv´)]
    sitesw_vec = [[w] for w in sitesw]
    sites_separatew = [x for pair in zip(sitesvv´_vec, sitesw_vec) for x in pair]
    full_vv´_w = Quantics.rearrange_siteinds(full_mps, sites_separatew)
    chi0_vv´_w = Quantics.rearrange_siteinds(chi0_mps, sites_separatew)
    gamma_vv´_w = Quantics.rearrange_siteinds(gamma_mps, sites_separatew)

    full_vv´_ww´ = Quantics.makesitediagonal(full_vv´_w, "w")
    chi0_vv´_ww´ = Quantics.makesitediagonal(chi0_vv´_w, "w")
    gamma_vv´_ww´ = Quantics.makesitediagonal(gamma_vv´_w, "w")
    diagonal_sites = full_vv´_ww´.sites

    chi0_vv´_w´w´´ = prime(chi0_vv´_ww´)
    gamma_vv´_w´´w´´´ = prime(gamma_vv´_ww´, 2)

    full_ptt = TCIA.ProjTTContainer{Float64}(full_vv´_ww´)
    chi0_ptt = TCIA.ProjTTContainer{Float64}(chi0_vv´_w´w´´)
    gamma_ptt = TCIA.ProjTTContainer{Float64}(gamma_vv´_w´´w´´´)

    pttfuncs = (; full_ptt, chi0_ptt, gamma_ptt)

    return pttfuncs, diagonal_sites
end;

In [8]:
function calculatebse(pttfuncs, diagonal_sites, maxbonddim, sites)
    (; full_ptt, chi0_ptt, gamma_ptt) = pttfuncs
    pordering = TCIA.PatchOrdering(collect(eachindex(diagonal_sites)))

    chi0_gamma_ptt = TCIA.adaptivematmul(chi0_ptt, gamma_ptt, pordering;
                                         maxbonddim)
    phi_bse_diagonal = TCIA.adaptivematmul(full_ptt, chi0_gamma_ptt, pordering;
                                           maxbonddim)

    phi_bse_diagonal_projmps = TCIA.ProjMPSContainer(Float64, phi_bse_diagonal,
                                                     diagonal_sites)
    phi_bse_projmps_vv´_w = Quantics.extractdiagonal(phi_bse_diagonal_projmps, "w")
    phi_bse_projmps_vv´w = Quantics.rearrange_siteinds(phi_bse_projmps_vv´_w,
                                                       sites.sitesfused)
    phi_bse = TCIA.ProjTTContainer{Float64}(phi_bse_projmps_vv´w)

    return phi_bse
end;

In [9]:
function comparereference(phi_bse, plainfuncs, grid)
    N = 2^(grid.R)
    vv = range(-N + 1; step=2, length=N)
    v´v´ = range(-N + 1; step=2, length=N)
    ww = range(-N; step=2, length=N)
    box = [(v, v´, w) for v in vv, v´ in v´v´, w in ww]

    (; fq_full, fq_chi0, fq_gamma) = plainfuncs
    bse_formula(v, v´, w) = sum(fq_full(v, v´´, w) *
                                fq_chi0(v´´, v´´´, w) *
                                fq_gamma(v´´´, v´, w) for v´´ in vv, v´´´ in vv)
    phi_normalmul = map(splat(bse_formula), box)

    phi_adaptivemul = [phi_bse(QG.origcoord_to_quantics(grid, p)) for p in box]

    error = norm(phi_normalmul - phi_adaptivemul, Inf) / norm(phi_normalmul, Inf)
    return error
end;

In [ ]:
ch_d = DensityChannel()
ch_m = MagneticChannel()
ch_s = SingletChannel()
ch_t = TripletChannel()
channels = (ch_d, ch_m, ch_s, ch_t)

println("Channel", "\t\t\t", "Error")
for ch in channels
    error = main(3.0, 10.0, ch, 4, 40)
    println(ch, "\t", error)
end

Channel			Error
DensityChannel()	4.578295295493737e-14
MagneticChannel(

Results from our implementation are up to floating point accuracy identical to the reference.

# Scaling analysis

To see how the number of patches we create depends on $R$ and on $D_\mathrm{max}$, we set $U=1$ and $\beta = 1.3$ and `adaptiveinterpolate` the full vertex in the density channel $F^{\mathrm{d}}$.

In [ ]:
using CairoMakie          # plotting library

In [ ]:
function numpatches(R, maxbonddim, tolerance=1e-8)
    grid, sites = setup(R)

    U = 1.0
    beta = 1.3
    ch = DensityChannel()
    _, quanticsfuncs = makeverts(U, beta, ch, grid)

    localdims = dim.(sites.sitesfused)
    projectable_full = TCIA.makeprojectable(Float64, quanticsfuncs.fI_full,
                                            localdims)

    initialpivots = [QG.origcoord_to_quantics(grid, 0)]
    full_patches = TCIA.adaptiveinterpolate(projectable_full;
                                            maxbonddim, initialpivots, tolerance)

    sitedims = [dim.(s) for s in sites.sitesfused]
    full_patches = reshape(full_patches, sitedims)
    return length(full_patches.data)
end;

First, we fix $D_\mathrm{max} = 30$ and vary $R$ from $2$ to $10$.

<!--

In [ ]:
Rs = 2:10
R_npatches = numpatches.(Rs, 30)
xlabel = L"Meshsize $R$"
ylabel = L"Number of patches in $F^{\mathrm{d}}$"
title = L"Tolerance = $10^{-8}$"
axis=(; xlabel, ylabel, yscale=log10, title)

-->

In [ ]:
scatter(Rs, R_npatches; axis)

The number of patches created seems to go roughly exponentially with $R$ at the start and level off at higher values.
To try and see if it saturates eventually, we increase the tolerance used in interpolating the vertex from the default value $10^{-8}$ to $10^{-4}$, enabling us to compute higher $R$ values in a reasonable time.

<!--

In [ ]:
R_hightols = 2:18
R_hightol_npatches = numpatches.(R_hightols, 30, 1e-4);
xlabel = L"Meshsize $R$"
ylabel = L"Number of patches in $F^{\mathrm{d}}$"
title = L"Tolerance = $10^{-4}$"
axis=(; xlabel, ylabel, yscale=log10, title)

-->

In [ ]:
scatter(R_hightols, R_hightol_npatches; axis)

While not completely saturating, the growth rate of the number of patches continues to slow.

Next, we fix $R = 6$ and vary $D_\mathrm{max}$ from $10$ to $120$.

<!--

In [ ]:
maxbonddims = 10:2:120
maxbonddim_npatches = numpatches.(6, maxbonddims);
xlabel = L"Max Bond Dimension $D_\mathrm{max}$"
ylabel = L"Number of patches in $F^{\mathrm{d}}$"
axis=(; xlabel, ylabel)

-->

In [ ]:
scatter(maxbonddims, maxbonddim_npatches; axis)

The number of patches decreases linearly with increasing bond dimension $D_\mathrm{max}$ before reaching 8 at $D_\mathrm{max} = 65$.
The bond dimension necessary to cover the entire box with a single tensor train at the requested accuracy of $10^{-8}$ is $D = 267$, so:

In [ ]:
@show numpatches(6, 266) numpatches(6, 267);

<!--